In [1]:
import numpy as np
import pandas as pd
try:
  from ucimlrepo import fetch_ucirepo
except Exception:
  !pip install ucimlrepo
  from ucimlrepo import fetch_ucirepo
import warnings
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error , r2_score

pd.pandas.set_option('display.max_columns', None)
%matplotlib inline
warnings.filterwarnings('ignore')

c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df_train = pd.read_csv('train_concrete.csv')
df_test = pd.read_csv('test_concrete.csv')
df_dev = pd.read_csv('dev_concrete.csv')

In [86]:
X_train = df_train.iloc[:, :-1].values
X_test = df_test.iloc[:, :-1].values
X_dev = df_dev.iloc[:, :-1].values
y_train = df_train.iloc[:, -1].values.reshape(-1,1)
y_test = df_test.iloc[:, -1].values.reshape(-1,1)
y_dev = df_dev.iloc[:, -1].values.reshape(-1,1)

In [87]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        
        # for leaf node
        self.value = value

In [88]:
class MyDecisionTreeRegressor():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree '''
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["var_red"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["var_red"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_var_red = -float("inf")
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    # update the best split if needed
                    if curr_var_red>max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def variance_reduction(self, parent, l_child, r_child):
        ''' function to compute variance reduction '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        val = np.mean(Y)
        return val
                
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        # Reshape Y to ensure it's one-dimensional
        Y = np.squeeze(Y)
        
        dataset = np.concatenate((X, Y.reshape(-1, 1)), axis=1)
        self.root = self.build_tree(dataset)
        
    def make_prediction(self, x, tree):
        ''' function to predict new dataset '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    def predict(self, X):
        ''' function to predict a single data point '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

In [89]:
regressor = MyDecisionTreeRegressor(min_samples_split=2, max_depth=3)
regressor.fit(X_train,y_train)

In [90]:
y_pred = regressor.predict(X_dev)
rmse = np.sqrt(mean_squared_error(y_dev, y_pred))
r2 = r2_score(y_dev, y_pred)
print("Scratch Implementation")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R2 Score: {r2:.2f}")

Scratch Implementation
Root Mean Squared Error: 0.63
R2 Score: 0.62


In [91]:
# Define different values for max_depth to test
max_depth_values = [2, 3, 5]

print("Decision Tree Regressor Results for Different max_depth Values:")
for max_depth in max_depth_values:
    print(f"max_depth={max_depth}")

    # Initialize and train the DecisionTreeRegressor
    model = DecisionTreeRegressor(max_depth=max_depth)
    model.fit(X_train, y_train)

    # Make predictions on the development set
    y_pred = model.predict(X_dev)

    # Calculate RMSE and R2 score
    rmse = mean_squared_error(y_dev, y_pred, squared=False)
    r2 = r2_score(y_dev, y_pred)

    print(f"Root Mean Squared Error: {rmse:.2f}")
    print(f"R2 Score: {r2:.2f}\n")


Decision Tree Regressor Results for Different max_depth Values:
max_depth=2
Root Mean Squared Error: 0.76
R2 Score: 0.44

max_depth=3
Root Mean Squared Error: 0.68
R2 Score: 0.56

max_depth=5
Root Mean Squared Error: 0.56
R2 Score: 0.70



In [92]:
class MyGradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, min_samples_split=2, max_depth=2):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        # Initialize the residuals as y
        residuals = np.copy(y).reshape(-1)  # Reshape to ensure it's one-dimensional

        for _ in range(self.n_estimators):
            # Fit a new tree to the residuals
            tree = MyDecisionTreeRegressor(min_samples_split=self.min_samples_split, max_depth=self.max_depth)
            tree.fit(X, residuals)

            # Compute predictions of the new tree
            tree_predictions = np.array(tree.predict(X)).reshape(-1)  # Reshape to ensure it's one-dimensional

            # Update residuals by subtracting the predictions scaled by learning_rate
            residuals -= self.learning_rate * tree_predictions

            # Store the new tree in the list of trees
            self.trees.append(tree)

    def predict(self, X):
        # Initialize predictions with zeros
        predictions = np.zeros(len(X))

        # Add predictions of each tree scaled by learning_rate
        for tree in self.trees:
            predictions += self.learning_rate * np.array(tree.predict(X))

        return predictions


In [93]:
gb_regressor = MyGradientBoostingRegressor(n_estimators=100, learning_rate=0.1)
gb_regressor.fit(X_train, y_train)

In [95]:
GBY_pred = gb_regressor.predict(X_dev)
rmse = np.sqrt(mean_squared_error(y_dev, GBY_pred))
r2 = r2_score(y_dev, GBY_pred)
print("Scratch Implementation")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R2 Score: {r2:.2f}")

Scratch Implementation
Root Mean Squared Error: 0.36
R2 Score: 0.87


In [96]:
# Define different values for n_estimators to test
n_estimators_values = [50, 100, 150]

print("Gradient Boosting Regressor Results for Different n_estimators Values:")
for n_estimators in n_estimators_values:
    print(f"n_estimators={n_estimators}")

    # Initialize the GradientBoostingRegressor with desired parameters
    gb_regressor = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=0.1)

    # Train the model using the training data
    gb_regressor.fit(X_train, y_train)

    # Make predictions on the development set
    y_pred = gb_regressor.predict(X_dev)

    # Calculate RMSE and R2 score for the development set
    gb_rmse = mean_squared_error(y_dev, y_pred, squared=False)
    gb_r2 = r2_score(y_dev, y_pred)

    print(f"RMSE: {gb_rmse:.2f}")
    print(f"R-squared Score: {gb_r2:.2f}\n")


Gradient Boosting Regressor Results for Different n_estimators Values:
n_estimators=50
RMSE: 0.43
R-squared Score: 0.82

n_estimators=100
RMSE: 0.39
R-squared Score: 0.85

n_estimators=150
RMSE: 0.36
R-squared Score: 0.87

